In [13]:
import os

import pandas as pd

data_dir = "./data"
files = [f for f in os.listdir(data_dir)]

In [14]:
def calculateTop(res, n):
    return sorted(res, key=res.get, reverse=True)[:n]

In [15]:
ma_file = []
ma_win_rate = []
ey_file = []
ey_win_rate = []

for file_name in files:
    df = pd.read_csv(data_dir + '/' + file_name)
    group_by_winner_df = df.groupby(['winner'])['winner'].size().reset_index(name='round')

    if 'ma' in file_name:
        total = sum(group_by_winner_df['round'])
        if group_by_winner_df[group_by_winner_df['winner'] == 'MARQUISE'].empty:
            win = 0
        else:
            win = group_by_winner_df[group_by_winner_df['winner'] == 'MARQUISE'].iloc[0]['round']
        win_rate = win / total

        avg_turn = float("INF")
        if win > 0:
            avg_turn = sum(df[df['winner'] == 'MARQUISE']['turn']) / len(df[df['winner'] == 'MARQUISE']['turn'])

        ma_win_rate.append((win_rate, -avg_turn))
        ma_file.append(file_name)
    else:
        total = sum(group_by_winner_df['round'])
        if group_by_winner_df[group_by_winner_df['winner'] == 'EYRIE'].empty:
            win = 0
        else:
            win = group_by_winner_df[group_by_winner_df['winner'] == 'EYRIE'].iloc[0]['round']
        win_rate = win / total

        avg_turn = float("INF")
        if win > 0:
            avg_turn = sum(df[df['winner'] == 'EYRIE']['turn']) / len(df[df['winner'] == 'EYRIE']['turn'])

        ey_win_rate.append((win_rate, -avg_turn))
        ey_file.append(file_name)

ma_res = {ma_file[i]: ma_win_rate[i] for i in range(len(ma_file))}
ey_res = {ey_file[i]: ey_win_rate[i] for i in range(len(ey_file))}

print("Marquise Top 5")
for ind, top in enumerate(calculateTop(ma_res, 5)):
    if ind == 0:
        print(str(ind + 1) + "st: " + top)
    elif ind == 1:
        print(str(ind + 1) + "nd: " + top)
    elif ind == 2:
        print(str(ind + 1) + "rd: " + top)
    else:
        print(str(ind + 1) + "th: " + top)
    stats = ma_win_rate[ma_file.index(top)]
    print("\t|_ win rate: " + str(stats[0]))
    print("\t|_ avg turn: " + str(-stats[1]))

print()

print("Eyrie Top 5")
for ind, top in enumerate(calculateTop(ey_res, 5)):
    if ind == 0:
        print(str(ind + 1) + "st: " + top)
    elif ind == 1:
        print(str(ind + 1) + "nd: " + top)
    elif ind == 2:
        print(str(ind + 1) + "rd: " + top)
    else:
        print(str(ind + 1) + "th: " + top)
    stats = ey_win_rate[ey_file.index(top)]
    print("\t|_ win rate: " + str(stats[0]))
    print("\t|_ avg turn: " + str(-stats[1]))


Marquise Top 5
1st: config-86-ma-100-2024-02-24-01-23-54.csv
	|_ win rate: 0.61
	|_ avg turn: 16.78688524590164
2nd: config-76-ma-100-2024-02-23-13-31-14.csv
	|_ win rate: 0.55
	|_ avg turn: 15.418181818181818
3rd: config-85-ma-100-2024-02-24-01-23-54.csv
	|_ win rate: 0.53
	|_ avg turn: 15.547169811320755
4th: config-87-ma-100-2024-02-24-01-23-54.csv
	|_ win rate: 0.53
	|_ avg turn: 16.245283018867923
5th: config-89-ma-100-2024-02-24-01-23-55.csv
	|_ win rate: 0.52
	|_ avg turn: 15.326923076923077

Eyrie Top 5
1st: config-184-ey-100-2024-02-26-15-48-41.csv
	|_ win rate: 0.81
	|_ avg turn: 14.518518518518519
2nd: config-203-ey-100-2024-02-28-14-16-54.csv
	|_ win rate: 0.72
	|_ avg turn: 16.72222222222222
3rd: config-194-ey-100-2024-02-26-15-48-42.csv
	|_ win rate: 0.71
	|_ avg turn: 15.690140845070422
4th: config-212-ey-100-2024-02-28-14-16-54.csv
	|_ win rate: 0.71
	|_ avg turn: 16.676056338028168
5th: config-202-ey-100-2024-02-28-14-16-54.csv
	|_ win rate: 0.71
	|_ avg turn: 16.80281

In [16]:
import numpy as np

avg_win_rate_ma = np.average([wr[0] for wr in ma_win_rate if wr[0] != 0])
print(avg_win_rate_ma)
avg_win_rate_ey = np.average([wr[0] for wr in ey_win_rate if wr[0] != 0])
print(avg_win_rate_ey)

avg_turn_to_win_ma = np.average([-wr[1] for wr in ma_win_rate if wr[1] != -float('inf')])
print(avg_turn_to_win_ma)
avg_turn_to_win_ey = np.average([-wr[1] for wr in ey_win_rate if wr[1] != -float('inf')])
print(avg_turn_to_win_ey)

0.24390243902439024
0.3593333333333333
18.12310918096633
16.418935623261522


In [17]:
import plotly.figure_factory as ff

def update_fig(figure):
    figure.update_layout(
        plot_bgcolor='white',
        paper_bgcolor="white"
    )
    figure.update_xaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey'
    )
    figure.update_yaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey'
    )
    figure.update_layout(font=dict(color='#000000'))


In [18]:
group_labels = ['ey_win_rate', 'ma_win_rate']
fig = ff.create_distplot([[data[0] for data in ey_win_rate], [data[0] for data in ma_win_rate]], group_labels,
                         bin_size=[0.1, 0.1], show_hist=True, show_rug=False, curve_type='normal',
                         histnorm='probability').update_layout(
    xaxis_title="Win rate", yaxis_title="Probability"
)
update_fig(fig)
fig.show()
fig.write_image("images/fig-stage-1-win-rate-dist.jpeg", scale = 6)

group_labels = ['ey_avg_turn', 'ma_avg_turn']
fig = ff.create_distplot([[-data[1] for data in ey_win_rate if data[1] != -float('inf')],
                          [-data[1] for data in ma_win_rate if data[1] != -float('inf')]], group_labels,
                         bin_size=[1, 1], show_hist=True, show_rug=False, curve_type='normal',
                         histnorm='probability density').update_layout(
    xaxis_title="Average turn", yaxis_title="Density"
)
update_fig(fig)
fig.show()
fig.write_image("images/fig-stage-1-avg-turn-dist.jpeg", scale = 6)